In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer

pd.set_option('display.max_columns', None)

In [3]:
R = pd.read_csv("../data/R.csv") #From DB5 dataset | log data
subset_ids = pd.read_csv("../data/R_ids_alpha_0.6556.csv")
data = R[R.id.isin(subset_ids.id.values)]

In [4]:
data.columns

Index(['ind', 'id', 'PHASE', 'TOK', 'IP', 'BT', 'NEL', 'PLTH', 'RGEO', 'KAREA',
       'EPS', 'MEFF', 'TAUTH', 'DATE', 'SHOT', 'TIME', 'Q95', 'ZEFF', 'AMIN',
       'VOL', 'POHM', 'PNBI', 'DWDIA', 'DWMHD', 'PICRH', 'PECRH', 'PL',
       'PFLOSS', 'TAV', 'LCOULOMB', 'QCYL5', 'TAUBOHM', 'RHOSTAR', 'BETASTAR',
       'NUSTAR', 'OMEGACYCL'],
      dtype='object')

In [5]:
not_useful_cols = ['ind', 'id', 'PHASE', 'DATE', 'SHOT']
data_cols = pd.Series(data.columns)
data = data[data_cols[~data_cols.isin(not_useful_cols)].values]
data.head(95)

,TOK,IP,BT,NEL,PLTH,RGEO,KAREA,EPS,MEFF,TAUTH,TIME,Q95,ZEFF,AMIN,VOL,POHM,PNBI,DWDIA,DWMHD,PICRH,PECRH,PL,PFLOSS,TAV,LCOULOMB,QCYL5,TAUBOHM,RHOSTAR,BETASTAR,NUSTAR,OMEGACYCL
0,AUG,-0.002904,0.919486,1.962908,1.488400,0.509825,0.464991,-1.206780,0.693147,-2.409169,3.40,1.380528,NaN,0.4981,12.98,400100.0,4981000.0,3578.0,17380.0,0.0,0.0,5320000.0,887000.0,1020.565150,15.028261,1.103085,0.112722,-5.257599,-0.198716,-1.282967,1.2540
8,AUG,-0.228030,0.653886,2.013836,1.796747,0.509224,0.476234,-1.212624,0.405465,-2.961524,2.55,1.345212,NaN,0.4949,12.95,425000.0,7078000.0,559.5,-34700.0,0.0,0.0,7450000.0,1415000.0,761.312828,14.709729,1.060839,0.066331,-5.275929,0.090343,-0.701408,1.2820
20,AUG,0.179818,1.089908,2.244002,1.202972,0.512824,0.479954,-1.231289,0.693147,-2.084253,2.25,1.316676,NaN,0.4875,12.66,757200.0,2907000.0,NaN,-85120.0,0.0,0.0,3750000.0,426000.0,820.096900,14.669024,1.057740,0.184983,-5.515855,-0.477155,-0.594271,1.4870
21,AUG,0.179818,1.089908,2.276344,1.193922,0.511625,0.473747,-1.226609,0.693147,-2.096384,3.35,1.315872,NaN,0.4892,12.65,850900.0,2872000.0,NaN,-9415.0,0.0,0.0,3740000.0,442300.0,779.255665,14.601770,1.059743,0.182752,-5.544878,-0.495897,-0.470573,1.4870
25,AUG,-0.225521,0.704087,1.755614,0.797507,0.505612,0.478716,-1.209820,0.693147,-2.077044,3.00,1.394511,NaN,0.4945,12.92,339000.0,2067000.0,7805.0,11510.0,0.0,0.0,2400000.0,170400.0,882.464134,14.986514,1.112370,0.126678,-5.107644,-0.120606,-1.192622,1.0110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,AUG,0.001000,0.939617,1.238374,1.460938,0.503801,0.478096,-1.220217,0.693147,-1.838851,2.35,1.444327,3.043,0.4885,12.57,139900.0,4818000.0,NaN,16470.0,0.0,0.0,4940000.0,440100.0,2641.662553,16.341580,1.101229,0.203440,-4.782743,-0.012459,-3.813549,1.2795
309,AUG,0.001000,0.938835,1.329195,1.619388,0.503801,0.476234,-1.219398,0.693147,-1.976885,3.15,1.444563,2.870,0.4889,12.57,118200.0,5840000.0,200400.0,206400.0,0.0,0.0,5750000.0,492400.0,2573.338155,16.269965,1.100244,0.177072,-4.795881,0.053721,-3.676924,1.2785
310,AUG,0.001000,0.938444,1.602816,1.905088,0.505612,0.489193,-1.230044,0.693147,-2.088280,3.85,1.448095,2.164,0.4846,12.53,109700.0,7282000.0,NaN,21760.0,0.0,0.0,7370000.0,534300.0,2577.533134,16.134783,1.093788,0.158344,-4.785842,0.329753,-3.403580,1.2780
311,AUG,0.001000,0.938444,1.617604,1.930071,0.506215,0.491643,-1.232299,0.693147,-2.143873,4.20,1.447389,2.143,0.4838,12.53,121100.0,7295000.0,NaN,-113000.0,0.0,0.0,7530000.0,529700.0,2488.722083,16.092325,1.092301,0.149782,-4.801721,0.309478,-3.318802,1.2780


In [6]:
data.ZEFF.unique()

array([       nan, 1.638     , 1.533     , 1.496     , 2.196     ,
       2.408     , 2.534     , 2.619     , 1.782     , 1.748     ,
       1.76      , 1.738     , 1.641     , 2.709     , 1.589     ,
       3.263     , 2.122     , 1.96      , 1.626     , 1.616     ,
       1.613     , 1.614     , 1.507     , 1.847     , 1.489     ,
       1.993     , 3.909     , 4.375     , 4.095     , 4.617     ,
       5.206     , 3.326     , 3.647     , 3.529     , 3.4       ,
       3.165     , 2.933     , 2.741     , 2.679     , 2.496     ,
       2.443     , 2.386     , 2.155     , 2.106     , 4.753     ,
       3.191     , 3.466     , 3.562     , 3.799     , 3.528     ,
       3.55      , 3.349     , 2.004     , 2.084     , 3.089     ,
       1.453     , 1.494     , 1.412     , 1.391     , 1.385     ,
       1.73      , 1.449     , 1.36      , 1.56      , 2.652     ,
       1.879     , 2.091     , 3.194     , 3.339     , 3.057     ,
       3.163     , 3.043     , 2.87      , 2.164     , 2.143  